<a href="https://colab.research.google.com/github/OzanGenc/CocktailAnalysis/blob/main/CocktailRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary Packages

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize
import nltk
from nltk.stem.porter import PorterStemmer

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Reading and Exploring the Data

In [3]:
cocktails_df = pd.read_csv('cocktails.csv')

In [4]:
cocktails_df.drop(columns=['Bartender', 'Location', 'Bar/Company', 'Glassware', 'Notes'], inplace=True)

In [5]:
cocktails_df

,Cocktail Name,Ingredients,Garnish,Preparation
0,Flor de Amaras,"1.5 oz Mezcal, 1 oz Hibiscus Simple Syrup*, .5...",Marigold Petals,*Hibiscus Simple Syrup:\n1:1 w/ a cup of dried...
1,The Happy Place,"2 oz Junipero Gin, .75 oz House-made Cranberry...","Dehydrated Lemon Wheel, Sprig of Rosemary",*House-made Cranberry syrup: \n-- 2 cups Fresh...
2,Bon Voyage Pisco Punch,"1500 ml BarSol Selecto Italia Pisco, 750 ml Le...",NaN,*Pineapple Gomme: \nMix equal parts (1.5 cups)...
3,Still Life of a Pineapple,"1.5 oz BarSol Primero Quebranta Pisco, .75 oz ...",NaN,*Pineapple Syrup:\n<em>Equal parts pineapple b...
4,The Bittered Valley,"1.25 oz Luxardo Maraschino Liqueur, 4 drops Ac...",NaN,"1st glass ingredients:\nLuxardo Maraschino, Ac..."
...,...,...,...,...
682,The Negroni Bianco,"1.5 oz Luxardo Bitter Bianco, .75 oz Tempus Fu...",NaN,NaN
683,Match Grip Julep,"1.5 oz H by HINE, .5 oz Jamaican Rum, 1 tsp Ca...",NaN,NaN
684,The Dry Cruise,"1 oz Luxardo Bitter Bianco, 1 oz Lustau Oloros...","Edible Flower, Straw",NaN
685,Red Hook,"2 oz Rye Whiskey, 5 oz Punt e Mes, .5 oz Luxar...",NaN,NaN


In [6]:
cocktails_df['Cocktail Name'].describe()

count         687
unique        684
top       Negroni
freq            2
Name: Cocktail Name, dtype: object

# Preprocessing 

In [7]:
cocktails_df.drop_duplicates(subset='Cocktail Name', inplace=True)

In [8]:
cocktails_df.fillna('', inplace=True)

In [9]:
cocktails_df['All Ingredients'] = cocktails_df['Ingredients'] + ',' + cocktails_df['Garnish']

In [10]:
cocktails_df['All Ingredients'].head()

0    1.5 oz Mezcal, 1 oz Hibiscus Simple Syrup*, .5...
1    2 oz Junipero Gin, .75 oz House-made Cranberry...
2    1500 ml BarSol Selecto Italia Pisco, 750 ml Le...
3    1.5 oz BarSol Primero Quebranta Pisco, .75 oz ...
4    1.25 oz Luxardo Maraschino Liqueur, 4 drops Ac...
Name: All Ingredients, dtype: object

In [11]:
additional_stop_words = frozenset(['oz', 'simple'])

cocktail_stop_words = ENGLISH_STOP_WORDS.union(additional_stop_words)

# Vectorizing text data with Tfidf Technique

In [12]:
vectorizer = TfidfVectorizer(stop_words=cocktail_stop_words, token_pattern=r'\b[^\d\W][^\d\W]+\b')

tfidf_matrix = vectorizer.fit_transform(cocktails_df['All Ingredients'])

In [13]:
cocktail_feature_df = pd.DataFrame(tfidf_matrix.toarray() ,columns=vectorizer.get_feature_names(), index=cocktails_df['Cocktail Name'])

In [26]:
cocktails_df[cocktails_df['Cocktail Name'] == 'Flor de Amaras']['Ingredients']

0    1.5 oz Mezcal, 1 oz Hibiscus Simple Syrup*, .5...
Name: Ingredients, dtype: object

# Calculating Similarities

In [15]:
similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

In [17]:
similarity_df = pd.DataFrame(similarity_matrix, columns=cocktail_feature_df.index, index=cocktail_feature_df.index)

In [18]:
similarity_df

Cocktail Name,Flor de Amaras,The Happy Place,Bon Voyage Pisco Punch,Still Life of a Pineapple,The Bittered Valley,OH Snap!,Wabash Avenue Sour,IPA Mule,Ritz Cocktail,King's Snap the Chocolate Out of You,Happiness,Fourth Generation,West Coast Punch,Mille Miglia,Paradiso Sun,Lux Orchard Lemonade,Cut & Rum,The King's Spring Daisy,The King's Ginger Spiced Punch,War of the Roses,The King's Medicine,A.Nise,Down the Rabbit Hole,Peace Daiquiri,The King's Autumn Cobbler,The Mad King,The Robert Baratheon,Lux Shandy,Fiona Graham,Le Sang et Sable,Martinez,Amaretto Sour BV-Style,Julia's Child,The Bitter King,La Maracuja,San Francisco Strong (Skrong),Nuestra Julep,Pisco Punch,La Batida Loca,Neighborhood Negroni,...,California Aurora,Plum Loco,Port's North Fogcutter Cup,Mayan Sacrifice,"Lox, Stock & Two Smoking Barrels",Speyside Highball,The Shah's Shandy,Elderfashion,Lychee & 3,Mauritian Sunrise,My Spicy Valentine,Five Star Cocktail,Doitashimashie,Lucky No.3,Motel Nacional,Southside Fizz,Champagne Sidecar,Pisco Sling,T&T,Scottish Fruit Tart,The Greatest Punch on Earth,3 & Tree Tonic,The King's Ginger Truffles,Kerouac,Cove,Margarita,Paloma,Margarita Ahumeada,Yuzu Sazerac,An Oaxacan Christmas,Im(pear)ed Fashioned,Spring Fling Sipper,Potrero Hill,No3,No.10,The Negroni Bianco,Match Grip Julep,The Dry Cruise,Red Hook,Union Jack
Cocktail Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Flor de Amaras,1.000000,0.031238,0.023366,0.017746,0.000000,0.033848,0.033755,0.055346,0.018481,0.014778,0.035091,0.080317,0.122642,0.117172,0.047905,0.016455,0.000000,0.040948,0.016887,0.028255,0.014586,0.024090,0.035937,0.105807,0.000000,0.024869,0.000000,0.050431,0.000000,0.027979,0.000000,0.056071,0.028952,0.080496,0.015423,0.031374,0.026106,0.083864,0.013246,0.000000,...,0.024466,0.010325,0.007973,0.000000,0.013650,0.087095,0.071945,0.022854,0.021683,0.022497,0.000000,0.049107,0.025831,0.000000,0.258078,0.143697,0.051615,0.054749,0.070494,0.014718,0.000000,0.121391,0.013436,0.091893,0.000000,0.175786,0.124920,0.125540,0.016099,0.113760,0.019025,0.182701,0.040400,0.014491,0.000000,0.000000,0.000000,0.029887,0.000000,0.000000
The Happy Place,0.031238,1.000000,0.051901,0.009997,0.019106,0.149595,0.056621,0.023078,0.045613,0.008325,0.072376,0.022656,0.081812,0.024069,0.061019,0.117135,0.000000,0.130770,0.257570,0.106368,0.055563,0.063533,0.091274,0.057332,0.017558,0.065235,0.029040,0.099475,0.028014,0.069055,0.033847,0.052299,0.090077,0.107504,0.039218,0.079685,0.039563,0.045442,0.007462,0.045451,...,0.040971,0.098888,0.073661,0.000000,0.007690,0.057901,0.061755,0.039049,0.027305,0.025347,0.028497,0.086420,0.044135,0.032503,0.054674,0.045392,0.068932,0.029256,0.000000,0.016583,0.018066,0.045227,0.007569,0.000000,0.009914,0.017391,0.017710,0.026870,0.009069,0.032089,0.077414,0.109616,0.095890,0.035766,0.000000,0.054257,0.008947,0.039915,0.000000,0.000000
Bon Voyage Pisco Punch,0.023366,0.051901,1.000000,0.120165,0.080152,0.039395,0.091139,0.017263,0.037005,0.008752,0.069732,0.011957,0.076200,0.007261,0.008819,0.047536,0.000000,0.138137,0.033814,0.059192,0.049773,0.076894,0.041826,0.038220,0.018458,0.053669,0.015327,0.050380,0.009870,0.056024,0.059303,0.066134,0.054507,0.093688,0.009134,0.036515,0.020880,0.355535,0.007845,0.000000,...,0.067339,0.015268,0.015964,0.000000,0.008084,0.051867,0.046739,0.027349,0.012841,0.101471,0.015040,0.061616,0.030911,0.010671,0.087980,0.022299,0.060073,0.133047,0.000000,0.008716,0.053503,0.026999,0.007957,0.000000,0.010422,0.009141,0.009309,0.008106,0.009534,0.016936,0.109467,0.051357,0.040359,0.029017,0.000000,0.000000,0.000000,0.034785,0.000000,0.000000
Still Life of a Pineapple,0.017746,0.009997,0.120165,1.000000,0.225545,0.083897,0.065916,0.013111,0.000000,0.011178,0.010569,0.000000,0.044736,0.000000,0.030519,0.000000,0.000000,0.018417,0.000000,0.012708,0.000000,0.044658,0.016163,0.020654,0.000000,0.018811,0.080337,0.016140,0.000000,0.000000,0.032558,0.049298,0.013022,0.036205,0.011666,0.014111

# Building Recommender Function

In [65]:
def cocktail_recommender(cocktail_name, num_recommendations=10, similarity_df=similarity_df, cocktails_df=cocktails_df):

  recommendations = similarity_df[cocktail_name].sort_values(ascending=False)[1:num_recommendations]
  recommendations.name = 'Similarity'

  cocktails_details = cocktails_df[cocktails_df['Cocktail Name'].isin(recommendations.index)].set_index('Cocktail Name')

  recommendations_df = pd.concat([cocktails_details,recommendations], axis=1).sort_values(by='Similarity', ascending=False)

  return recommendations_df

In [70]:
mycocktail = cocktail_recommender('Paloma')

In [71]:
mycocktail

,Ingredients,Garnish,Preparation,All Ingredients,Similarity
Motel Nacional,"1.5 oz Chinaco Blanco Tequila, .5 oz Luxardo A...",,,"1.5 oz Chinaco Blanco Tequila, .5 oz Luxardo A...",0.522972
Kerouac,"1.5 oz Chinaco Reposado Tequila, .5 oz Gran Cl...",Grapefruit Twist,,"1.5 oz Chinaco Reposado Tequila, .5 oz Gran Cl...",0.472251
Santa Patricia,"1 oz Blanco Tequila, .5 oz Dry Curacao, .5 oz ...",Fresh Grapefruit Slice,Shake all ingredients very well with ice and s...,"1 oz Blanco Tequila, .5 oz Dry Curacao, .5 oz ...",0.463760
Tamaulipas Margarita,"2 oz Chinaco Blanco Tequila, 1.5 oz Fresh Lime...",Lime Wheel,"Served in v-shaped rocks glass with ""vintage i...","2 oz Chinaco Blanco Tequila, 1.5 oz Fresh Lime...",0.454988
Hemingway Daiquiri,"1.5 oz Pink Pigeon Rum, .75 oz Fresh Lime Juic...",Grapefruit Twist,Shake with ice in a mixing tin and double stra...,"1.5 oz Pink Pigeon Rum, .75 oz Fresh Lime Juic...",0.437402
Salty Swede,"2 oz Karlsson's Gold Vodka, 2 oz Fresh grapefr...",,Combine the Karlsson's Gold Vodka and grapefru...,"2 oz Karlsson's Gold Vodka, 2 oz Fresh grapefr...",0.420855
The Mayan Premonition,"1.5 oz Chinaco Blanco Tequila, .75 oz Luxardo ...","Fresh Ginger Slice, Luxardo Cherry",Fill up a stemmed wine glass with ice then pou...,"1.5 oz Chinaco Blanco Tequila, .75 oz Luxardo ...",0.417825
#106 Cocktail,"1 oz Chinaco Blanco Tequila, 1 oz Luxardo Bitt...",Sage,Shake and double strain into a coupe. Garnish ...,"1 oz Chinaco Blanco Tequila, 1 oz Luxardo Bitt...",0.350437
Donaji,"2 oz Mezcal, .5 oz Fresh Lemon Juice, .5 oz Fr...","Chapuline, Salt & Chile Pepper Rim*",Shaken and strained over ice in a rocks glass....,"2 oz Mezcal, .5 oz Fresh Lemon Juice, .5 oz Fr...",0.342399


In [ ]:
1.5 oz Chinaco Blanco Tequila, .5 oz Luxardo Apricot Liqueur, .75 oz Lime juice, 1 oz Fresh Pineapple juice, .5 oz Simple syrup, 1 oz Soda water

In [ ]:
1 1/2 oz White rum (or aged rum)
1/4 oz Apricot liqueur
1 oz Pineapple juice
1/2 oz Fresh lime juice
1/2 oz Simple syrup
Garnish: Lime wheel